# vob master

* 1. 指定目录/文件夹 遍历PDF文件，解析文件并获得文字，然后解析单词
* 2. 对照已学词汇表，GRE，TOFEL等词汇表进行统计，并排序指出其频度

In [ ]:
import os
import sys
import re
import time
import PyPDF4
import pdftotext
import ipywidgets as widgets

learnt={}

def load_list_file(file):
    s={}
    text_file = open(file, "r")
    lines = text_file.readlines()
    for line in lines:
        s[(line.rstrip('\n'))]=1
    text_file.close()
    return s

def load_learnt(file):
    learnt = load_list_file(file)
    print("Words learnt:%d"%len(learnt))
    return learnt
        
def load_gre(file):
    s={}
    global learnt
    #print("file:%s,dic:%s"%(file,gre))
    text_file = open(file, "r")
    lines = text_file.readlines()
    text_file.close()

    for i in range(len(lines)//2):
        line = lines[i*2].rstrip('\n')
        if line not in learnt:
            s[line]=1
            
    print("GRE Loaded %d Words"%len(s))
    return s

def load_ielts(file):
    s={}
    text_file = open(file, "r")
    lines = text_file.readlines()
    text_file.close()

    for line in lines:
        word = line.split()[0]
        w = re.findall('^[a-z]*',line)[0]
        #print(w)
        if w not in learnt:
            s[w]=1
            
    print("IELTS Loaded %d Words"%len(s))
    return s
            

def load_cet(file):
    s={}
    text_file = open(file, "r")
    lines = text_file.readlines()
    text_file.close()

    for line in lines:
        if len(line) < 5 : continue
        word = line.split()[0]
        w = re.findall('^[a-z]*',line)[0]
        #print(w)
        if w not in learnt:
            s[w]=1
            
    print("CET-4 Loaded %d Words"%len(s))
    return s



def getWordCount(data, summary):
    data=data.split()
    for i in data:
        if i in gre:
            if i not in summary:
                summary[i]=1
            else:
                summary[i]=summary[i]+1
        #print(i)
def load_pdf_pdf2text(file):
    txt = file+".txt"
    print("Reading PDF Using PDF2TEXT....")

    cmd = "pdftotext \"%s\" \"%s\""%(file,txt)   
    print("cmd:%s"%cmd)
    os.system(cmd)
    if os.path.exists(txt):
        print("txt file generated")
        text_file = open(txt,'r')
        lines = text_file.readlines()
        text_file.close()
        os.remove(txt) 
        text=''
        for line in lines:
            text+=line
        return text
    else: return ''
        
def load_pdf_pypdf(file): 
    pdfFileObj = open(file, 'rb')
    pdfReader = PyPDF4.PdfFileReader(pdfFileObj)
    numPages = pdfReader.numPages
    print("PDF Loaded %d Pages"%numPages)
    print("isEncrypted:%s"%pdfReader.isEncrypted)
    text=''
    print("Reading PDF Using PYPDF4....")
    for i in range(numPages):
        pageObj = pdfReader.getPage(i)
        text += pageObj.extractText()
        if len (text)<100 and i > 10 :
            print("Pypdf read nothing")
            return ''
    return text

def load_pdf(file):
    try:
        if os.path.exists(file): pass
    except OSError as err:
        print(err.reason)
        exit(1)
    #test pypdf4
    text = load_pdf_pypdf(file)
    if text == '':
        text = load_pdf_pdf2text(file)
    return read_text(text)

def load_pdfs(path):
    try:
        if os.path.exists(path): pass
    except OSError as err:
        print(err.reason)
        exit(1)
    files= os.listdir(path)
    s={}
    for file in files:
        if file[-4:].upper() == '.PDF' :
            print("%s"%file)
            s.update(load_pdf(path + '/' + file))
            
    print("files:%s"%files)
    return s

def read_text(lines):
    summary={}
    new_text = re.sub('\s+',' ',lines)
    getWordCount(new_text.strip(),summary)
    return summary


def load_pdf_old(file):
    pdfFile = file
    # check if the specified file exists or not
    try:
        if os.path.exists(pdfFile): pass
            #print("file found!")
    except OSError as err:
        print(err.reason)
        exit(1)

    pdfFileObj = open(pdfFile, 'rb')
    pdfReader = PyPDF4.PdfFileReader(pdfFileObj)

    numPages = pdfReader.numPages
    print("PDF Loaded %d Pages"%numPages)
    print("isEncrypted:%s"%pdfReader.isEncrypted)
    summary={}

    for i in range(numPages):
    #for i in range(10):
        pageObj = pdfReader.getPage(i)
        text = pageObj.extractText()
        print("len(text):%d"%len(text))
        new_text = re.sub('\s+',' ',text)
        getWordCount(new_text.strip(),summary)
    return summary

def word_belong(word):
    belong=''
    if word in gre:
        belong+='GRE '
    if word in ielts:
        belong+='IELTS '
    if word in toefl:
        belong+='TOEFL '
    if word in cet4:
        belong+='CET-4 '
    if word in cet6:
        belong+='CET-6 '
    return belong

def show_summary(summary): 
    tmp=[];i=0
    for key, value in summary.items():
        tmptuple = (value, key)
        tmp.append(tmptuple)

    tmp = sorted(tmp,reverse=True)
    print("***************************summary*****************************")
    for v,k in tmp:
        print("%-20s %-40s %5s"%(k,word_belong(k),v))
        i+=1;
        if i> 20: break 
    return

def write_unlearn(summary):
    f=open("dict/unlearn.txt","w")
    for key, value in summary.items():
        f.write("%s\n"%key)
    f.close()

learnt = load_learnt("dict/learnt.txt")
gre   = load_gre("dict/gre.txt")
ielts = load_ielts("dict/ielts.txt")
cet4  = load_cet("dict/cet-4.txt")
cet6  = load_cet("dict/cet-6.txt")
toefl = load_cet("dict/toefl.txt")

summary = load_pdfs("pdf")
print("summary:%d"%len(summary))
show_summary(summary)
write_unlearn(summary)




In [ ]:
#lst_unlearn=list(summary.keys())
def load_unlearnt(file):
    s=[]
    unlearn = load_list_file(file)
    print("Words unlearnt:%d"%len(unlearn))
    for k,v in unlearn.items(): 
        s.append(k)
    return s

lst_unlearn=[]
lst_unlearn = load_unlearnt('dict/unlearn.txt')
num = 30

curr_index=0
check_learn={}



def check_unlearn():
    global curr_index
    
    if curr_index > len(lst_unlearn) :
        show_new_learn()
        return
    
    curr = curr_index
    #print("curr_index:%d"%curr_index)
    curr_index+=num
    
    create_tab(lst_unlearn[curr:curr+num])

def changed(b):
    #print(b)
    if b.owner.value:
        check_learn[b.owner.description]=1
    else:
        check_learn[b.owner.description]=0
    #print(">>>>%s<<<<"%b.owner.description)
        
def create_tab(data):
    boxes=[]
    hboxes=[]
    i=0
    #print("================")
    #print(data)
    for b in data:
        i+=1;
        box = widgets.Checkbox(description=b)
        #box.on_check(changed)
        box.observe(changed)
        boxes.append(box)
        if i % 5 ==0 :
            hboxes.append(widgets.HBox(boxes))
            i=0
            boxes.clear()
            
    vbox=widgets.VBox(hboxes)
            
    tab = widgets.Tab()
    tab.set_title(0, 'checking..')
    #text = widgets.Label( value='$Check$ $the$ $word$ $you$ $know$', )
    text = widgets.Label( value='Check word you know', )
    btn_next = widgets.Button(
        description='Next %d'%num,
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        #tooltip='OK'
        #icon='check'
    )
    btn_stop = widgets.Button(description='Abort',button_style = 'danger')
    h_btn = widgets.HBox([btn_next,btn_stop])
    panel = widgets.VBox([text,vbox,h_btn])
    tab.children = [panel]
    
    output = widgets.Output()
    
    display(tab,output)

    def on_button_clicked(b):
    #with output:
        tab.close()
        check_unlearn()
    def on_button_cancel(b):
        tab.close()
        show_new_learn()
    btn_next.on_click(on_button_clicked)
    btn_stop.on_click(on_button_cancel)


#} Show Tabs


def show_new_learn():
    learn=[]
    for k,v in check_learn.items():
        if v: 
            learn.append(k)
            lst_unlearn.remove(k)
    print(learn)
    save_learnt(learn)
    print("update unlearn:%d"%len(lst_unlearn))

def save_learnt(lst):
    f=open("dict/learnt.txt","a")
    for key in lst:
        f.write("%s\n"%key)
    f.close()
    
    f=open("dict/unlearn.txt","w+")
    f.seek(0)
    f.truncate()
    for key in lst_unlearn:
        f.write("%s\n"%key)
    f.close()

check_unlearn()
